# Calcul des différents critères de dynamiques

### *Ouverture des fichiers, chargement des librairies :*

In [25]:
# Creation des datasets

datacrop<-read.csv2("MAELIA_crop_raw.csv", dec=".", sep=";", header=T)
datalivestock<-read.csv2("MAELIA_livestock_raw.csv", dec=".", sep=";", header=T)
names(datacrop) <- c("Scenario", "Year", "Farm", "Parcel", "Crop", "Yield", "Area", "Revenue", "Variablecost", "Energy", "ProteinkgN", "PDIN", "ProteinN.ton", "Nitrogen", "Phosphorus", "Potassium", "Activeingredient")
names(datalivestock) <- c("Scenario", "Year", "Farm", "Milkrevenue", "Feedcost")

# Chargement des librairies

install.packages("tidyverse")
install.packages("tidyr")
install.packages("reshape2")
install.packages("cowplot")
install.packages("magrittr")
install.packages("ggplot2")
install.packages("corrplot")
install.packages("slider")

library(tidyverse)
library(reshape2)
library(cowplot)
library(ggplot2)
library(corrplot)
library(slider)
library(magrittr)

Warning message:
"package 'tidyverse' is in use and will not be installed"Warning message:
"package 'tidyr' is in use and will not be installed"Warning message:
"package 'reshape2' is in use and will not be installed"Warning message:
"package 'cowplot' is in use and will not be installed"

package 'magrittr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Utilisateur\AppData\Local\Temp\RtmpYjlnb4\downloaded_packages


Warning message:
"package 'ggplot2' is in use and will not be installed"

package 'corrplot' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Utilisateur\AppData\Local\Temp\RtmpYjlnb4\downloaded_packages
package 'slider' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Utilisateur\AppData\Local\Temp\RtmpYjlnb4\downloaded_packages


Warning message:
"package 'corrplot' was built under R version 3.6.3"corrplot 0.84 loaded
Warning message:
"package 'slider' was built under R version 3.6.3"Warning message:
"package 'magrittr' was built under R version 3.6.3"

ERROR: Error in value[[3L]](cond): Package 'magrittr' version 1.5 cannot be unloaded:
 Error in unloadNamespace(package) : namespace 'magrittr' is imported by 'tidyr', 'tibble', 'dplyr', 'forcats', 'rvest', 'stringr', 'modelr', 'dbplyr', 'tidyverse', 'purrr' so cannot be unloaded



### *Construction des fonctions de calcul de critères de dynamiques :*

#### Fonction 1. Variability :
Critères calculés :
* Slope of Finlay Wilkinson regression on detrended yield (code : slopeFW)
* Residuals of Linear Mixed Model (code : RESi)
* Stability around a moving average (code : MSV)
* Relative standard deviation (code : RSD)

References : 
* slopeFW : Li et al 2019, Macholdt et al 2020
* RESi : Martin et al 2017 
* MSV : Redhead et al 2020 
* RSD : Sneesens et al 2019

In [26]:
variability <- function(df, performances){
  for(i in performances){

    name <- colnames(df)[i]
    colnames(df)[i] <- "perf"

    df <- df%>%
      group_by(Farm)%>%
      mutate(detrend = if(all(is.na(perf))) NA else  resid(lm(perf ~ Year, na.action = na.exclude)))%>% # de-trended yield (residuals of linear regression)
      mutate(min = min(perf, na.rm = T), max = max(perf, na.rm = T), range = max - min)%>%  # yield range
      mutate(min = ifelse(min == Inf, NA, min), max = ifelse(max == -Inf, NA, max), range = ifelse(max == Inf, NA, range))%>%
      mutate(CV = sd(detrend, na.rm = T)/mean(perf, na.rm = T), VAR = var(detrend, na.rm = T))%>% # de-trended yield (residuals) cv and variance
      mutate(RSD = abs(sd(perf, na.rm = T)/mean(perf, na.rm = T))*100)%>% # relative standard deviation
      arrange(Farm,Year)%>%
      mutate(MVS = mean(100/abs(perf- slide_dbl(perf, mean, .before = 1, .after = 1)), na.rm = T))

    df <- df %>%
      group_by(Year)%>%
      mutate(EI = mean(detrend, na.rm = T)) # environmental index

    df <- df %>%
      group_by(Farm)%>%
      mutate(slopeFW = if(all(is.na(perf))) NA else abs(lm(detrend ~ EI, na.action = na.exclude)$coefficients[[2]]-1)) # slope of finlay-wilkinson regression

    colnames(df)[(ncol(df)-9):ncol(df)] <-c(paste0(name,".detrend"),paste0(name,".min"),
                                            paste0(name,".max"),paste0(name,".range"),
                                            paste0(name,".CV"),paste0(name,".VAR"), paste0(name, ".RSD"),
                                            paste0(name, ".MSV"),paste0(name,".EI"), paste0(name, ".slopeFW"))

    jpeg(paste0(name,".stability_cor.jpg"), width = 17, height = 17, units = "cm", res = 300)
    corrplot(cor(df[c((ncol(df)-6): (ncol(df) - 2), ncol(df))], use = "pairwise.complete.obs"))
    dev.off()

    colnames(df)[i] <- name

  }
  return(df)
}

#### Fonction 2. Resistance
Critères calculés :
* General resistance : Probability of crop failure (code : Probalow10)
* General resistance (other method) : Probability of crop failure (code : Probafail)
* Resistance to a particular event (drought) (code : RESe)
* Number of economic disruption (code : NED)
* Probability of peak work intensity (>80 percentile) (code : Probahigh)

References :
* Probalow10 & Probahigh : Li et al 2019
* Probafail : Zampieri et al 2020
* RESe : Redhead et al 2020
* NED : Sneessens et al 2019

In [27]:
resistance <- function(df, performances){
  for(i in performances){

    #df <- data_stab
    #i<- 10
    #attach(df)
    name <- colnames(df)[i]
    colnames(df)[i] <- "perf"
    colnames(df)[colnames(df) == paste0(name,".detrend")] <- "detrend"

    perclow <- quantile(df$perf, .10, na.rm = T)
    perchigh <- quantile(df$perf, .80, na.rm = T)

    df <- df%>%
      group_by(Farm)%>%
      mutate(probalow = length(na.omit(perf[perf < perclow]))/length(na.omit(perf)))%>%# probability of low performance level Li et al 2019
      mutate(probalow10 = dnorm((perclow - mean(perf, na.rm = T))/sd(perf, na.rm = T))*100)%>%# probability of low performance level Macholdt et al 2020
      mutate(probahigh = length(na.omit(perf[perf > perchigh]))/length(na.omit(perf)))%>%# probability of high performance level
      mutate(pi = if(all(is.na(perf)) | length(!is.na(perf)[!is.na(perf) == T]) == 1) NA else (fitted(loess(perf ~ Year, span = 0.85, na.action = na.exclude))))%>%
      mutate(loessdetrend = if(all(is.na(perf) | identical(round(perf,3),round(pi,3)))) NA else ((perf-pi)/pi))%>% # de trending with loess regression (y - fitted)/fitted
      mutate(probafail = 1/var(loessdetrend, na.rm = T))%>%# probability of failure/crash
      #mutate(RESe = if(length(perf[Year == 2015]) != 0) 1/(perf[Year == 2015] - mean(c(perf[Year == 2012],perf[Year == 2013],perf[Year == 2014]), na.rm = T)) else NA)%>% # Resistance to drought (2015)
      mutate(NED = length(na.omit(perf[perf < 0.75*slide_dbl(perf, mean, na.action = na.omit, .before = 2)])))%>% # number of economic disruption
      #mutate(RECOV = recovery(perf))# time to recover (in years) of economic disruption

    df <- df[,-ncol(df)-5]

    jpeg(paste0(name,".resist_cor.jpg"), width = 17, height = 17, units = "cm", res = 300)
    corrplot(cor(df[c((ncol(df)-7): (ncol(df)-5),(ncol(df)-3): ncol(df))], use = "pairwise.complete.obs"))
    #print(cor.test(pull(df[(ncol(df)-5)]), pull(df[ncol(df)-3])))
    dev.off()

    colnames(df)[(ncol(df)-7): ncol(df)] <- c(paste0(name, ".probalow"), paste0(name, ".probalow10"),
                                              paste0(name,".probahigh"),
                                              paste0(name, ".loessdetrend"),
                                              paste0(name, ".probafail"), paste0(name, ".RESe"),
                                              paste0(name, ".NED"), paste0(name, ".RECOV"))
    colnames(df)[i] <- name
    colnames(df)[colnames(df) == "detrend"] <- paste0(name,".detrend")
  }
  return(df)
}


#### Fonction 3. Level & Trend : 
Critères calculés : 
* Mean level (code : Level) 
* Mean relative distance to regional gross margin (code : RD)
* Slope of linear regression (code : Trend)

References : 
* Level : Martin et al 2017
* RD : Redhead et al 2020, SNeessens et al 2019
* Trend : Martin et al 2017

In [28]:

trend <- function(df, performances){
  for(i in performances){
    #df <- data_resist
    #i<- 10
    #i=10
    #df <- data_stab[data_stab$agroecosyst == "28",]
    #colnames(df)[i] <- "perf"
    #attach(df)
    name <- colnames(df)[i]

    colnames(df)[i] <- "perf"

    df <- df%>%
      group_by(Farm)%>%
      mutate(level = mean(perf, na.rm = T))%>%# mean level
      #mutate(RD = mean(RICA$gm[RICA$years %in% yr] - perf[yr %in% RICA$years] ,na.rm = T))%>%# mean relative distance to regional GM
      mutate(trend = if(all(is.na(perf))) NA else lm(perf ~ Year, na.action = na.exclude)$coefficients[[2]]) # slope of linear regression

    jpeg(paste0(name,".trend_cor.jpg"), width = 17, height = 17, units = "cm", res = 300)
    corrplot(cor(df[(ncol(df)-2): ncol(df)], use = "pairwise.complete.obs"))
    dev.off()

    colnames(df)[(ncol(df)-2): ncol(df)] <- c(paste0(name, ".level"), paste0(name, ".RD"), paste0(name, ".trend"))
    colnames(df)[i] <- name

    df <- df %>%
      mutate_at(vars(seq_len(ncol(df))), ~ifelse(. == "NaN", NA, .))
  }
  return(df)
}

### *Etude des différents critères sur des attributs de performances choisis :*

In [29]:
performances <- c(8, 9, 10, 13)
vardatacrop <- variability(datacrop, performances)

In [30]:
vardatacrop

Scenario,Year,Farm,Parcel,Crop,Yield,Area,Revenue,Variablecost,Energy,...,ProteinN.ton.detrend,ProteinN.ton.min,ProteinN.ton.max,ProteinN.ton.range,ProteinN.ton.CV,ProteinN.ton.VAR,ProteinN.ton.RSD,ProteinN.ton.MSV,ProteinN.ton.EI,ProteinN.ton.slopeFW
Baseline situation,2005,AF1,101_00,WheatW,7.48,2.18,861.2791,272.8,138.3297,...,113.74325,778.4,1516,737.6,0.1843649,41477.84,18.43655,Inf,-45.3997,1.055526
Baseline situation,2005,AF1,102_00,WheatW,7.48,1.51,861.2791,272.8,138.3297,...,113.74325,778.4,1516,737.6,0.1843649,41477.84,18.43655,Inf,-45.3997,1.055526
Baseline situation,2005,AF1,103_00,WheatW,7.48,5.65,861.2791,272.8,138.3297,...,113.74325,778.4,1516,737.6,0.1843649,41477.84,18.43655,Inf,-45.3997,1.055526
Baseline situation,2005,AF1,104_00,Fodder,7.46,4.36,285.5688,44.0,142.1736,...,-210.29675,778.4,1516,737.6,0.1843649,41477.84,18.43655,Inf,-45.3997,1.055526
Baseline situation,2005,AF1,104_00,gMaize,10.71,4.36,1267.8851,246.4,199.4073,...,-152.30675,778.4,1516,737.6,0.1843649,41477.84,18.43655,Inf,-45.3997,1.055526
Baseline situation,2005,AF1,104_01,Fodder,7.46,5.70,285.5688,44.0,142.1736,...,-210.29675,778.4,1516,737.6,0.1843649,41477.84,18.43655,Inf,-45.3997,1.055526
Baseline situation,2005,AF1,104_01,gMaize,10.71,5.70,1267.8851,246.4,199.4073,...,-152.30675,778.4,1516,737.6,0.1843649,41477.84,18.43655,Inf,-45.3997,1.055526
Baseline situation,2005,AF1,105_00,OSR,5.20,0.82,1345.2975,264.0,150.9922,...,-55.09675,778.4,1516,737.6,0.1843649,41477.84,18.43655,Inf,-45.3997,1.055526
Baseline situation,2005,AF1,106_00,Fodder,7.46,1.80,285.5688,44.0,142.1736,...,-210.29675,778.4,1516,737.6,0.1843649,41477.84,18.43655,Inf,-45.3997,1.055526
Baseline situation,2005,AF1,106_00,sMaize,16.92,1.80,1251.7410,246.4,313.0200,...,315.78325,778.4,1516,737.6,0.1843649,41477.84,18.43655,Inf,-45.3997,1.055526
